In [38]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler

import os

In [39]:
import matplotlib.pyplot as plt
import scipy
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [40]:
df = pd.read_csv("../input/gtzan-dataset-music-genre-classification/Data/features_3_sec.csv")
df.tail()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
9985,rock.00099.5.wav,66149,0.349126,0.080515,0.050019,0.000097,1499.083005,164266.886443,1718.707215,85931.574523,...,42.485981,-9.094270,38.326839,-4.246976,31.049839,-5.625813,48.804092,1.818823,38.966969,rock
9986,rock.00099.6.wav,66149,0.372564,0.082626,0.057897,0.000088,1847.965128,281054.935973,1906.468492,99727.037054,...,32.415203,-12.375726,66.418587,-3.081278,54.414265,-11.960546,63.452255,0.428857,18.697033,rock
9987,rock.00099.7.wav,66149,0.347481,0.089019,0.052403,0.000701,1346.157659,662956.246325,1561.859087,138762.841945,...,78.228149,-2.524483,21.778994,4.809936,25.980829,1.775686,48.582378,-0.299545,41.586990,rock
9988,rock.00099.8.wav,66149,0.387527,0.084815,0.066430,0.000320,2084.515327,203891.039161,2018.366254,22860.992562,...,28.323744,-5.363541,17.209942,6.462601,21.442928,2.354765,24.843613,0.675824,12.787750,rock
9989,rock.00099.9.wav,66149,0.369293,0.086759,0.050524,0.000067,1634.330126,411429.169769,1867.422378,119722.211518,...,38.801735,-11.598399,58.983097,-0.178517,55.761299,-6.903252,39.485901,-3.412534,31.727489,rock


In [41]:
# No of data items & no features present
df.shape

(9990, 60)

In [42]:
np.unique(df["label"], return_counts=True)

(array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
        'metal', 'pop', 'reggae', 'rock'], dtype=object),
 array([1000,  998,  997,  999,  998, 1000, 1000, 1000, 1000,  998]))

In [43]:
# Basic statistics about the dataset
df.describe()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
count,9990.0,9990.000000,9990.000000,9990.000000,9.990000e+03,9990.000000,9.990000e+03,9990.000000,9.990000e+03,9990.000000,...,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000
mean,66149.0,0.379534,0.084876,0.130859,2.676388e-03,2199.219431,4.166727e+05,2241.385959,1.182711e+05,4566.076592,...,1.448240,49.988755,-4.198706,51.962753,0.739943,52.488851,-2.497306,54.973829,-0.917584,57.322614
std,0.0,0.090466,0.009637,0.068545,3.585628e-03,751.860611,4.349644e+05,543.854449,1.013505e+05,1642.065335,...,5.735149,34.442816,5.677379,36.400669,5.181313,38.177120,5.111799,41.585677,5.253243,46.444212
min,66149.0,0.107108,0.015345,0.000953,4.379535e-08,472.741636,8.118813e+02,499.162910,1.183520e+03,658.336276,...,-26.850016,1.325786,-27.809795,1.624544,-20.733809,3.437439,-27.448456,3.065302,-35.640659,0.282131
25%,66149.0,0.315698,0.079833,0.083782,6.145900e-04,1630.680158,1.231961e+05,1887.455790,4.876553e+04,3378.311110,...,-2.227478,29.584894,-7.951722,29.863448,-2.516638,29.636197,-5.734123,30.496412,-4.004475,30.011365
50%,66149.0,0.384741,0.085108,0.121253,1.491318e-03,2208.628236,2.650692e+05,2230.575595,8.996072e+04,4631.377892,...,1.461623,41.702393,-4.443021,42.393583,0.733772,41.831377,-2.702366,43.435253,-1.030939,44.332155
75%,66149.0,0.442443,0.091092,0.176328,3.130862e-03,2712.581884,5.624152e+05,2588.340505,1.585674e+05,5591.634521,...,5.149752,59.274619,-0.726945,61.676964,3.888734,62.033906,0.514246,65.328602,2.216603,68.210421
max,66149.0,0.749481,0.120964,0.442567,3.261522e-02,5432.534406,4.794119e+06,3708.147554,1.235143e+06,9487.446477,...,39.144405,683.932556,34.048843,529.363342,36.970322,629.729797,31.365425,1143.230591,34.212101,910.473206


In [44]:
# Filename column is not needed and can be dropped
df = df.drop(labels='filename',axis=1)

In [45]:
# Imputation
from sklearn.impute import SimpleImputer


imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit_transform(df.iloc[:,:-1])

array([[ 6.61490000e+04,  3.35406363e-01,  9.10482928e-02, ...,
         3.36180725e+01, -2.43026793e-01,  4.37717667e+01],
       [ 6.61490000e+04,  3.43065351e-01,  8.61465260e-02, ...,
         9.70308304e+01,  5.78406334e+00,  5.99430809e+01],
       [ 6.61490000e+04,  3.46814752e-01,  9.22428891e-02, ...,
         5.30508347e+01,  2.51737475e+00,  3.31051216e+01],
       ...,
       [ 6.61490000e+04,  3.47480893e-01,  8.90194401e-02, ...,
         4.85823784e+01, -2.99545288e-01,  4.15869904e+01],
       [ 6.61490000e+04,  3.87527317e-01,  8.48154277e-02, ...,
         2.48436127e+01,  6.75824106e-01,  1.27877502e+01],
       [ 6.61490000e+04,  3.69292945e-01,  8.67586955e-02, ...,
         3.94859009e+01, -3.41253424e+00,  3.17274895e+01]])

In [46]:
df.describe()

,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
count,9990.0,9990.000000,9990.000000,9990.000000,9.990000e+03,9990.000000,9.990000e+03,9990.000000,9.990000e+03,9990.000000,...,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000,9990.000000
mean,66149.0,0.379534,0.084876,0.130859,2.676388e-03,2199.219431,4.166727e+05,2241.385959,1.182711e+05,4566.076592,...,1.448240,49.988755,-4.198706,51.962753,0.739943,52.488851,-2.497306,54.973829,-0.917584,57.322614
std,0.0,0.090466,0.009637,0.068545,3.585628e-03,751.860611,4.349644e+05,543.854449,1.013505e+05,1642.065335,...,5.735149,34.442816,5.677379,36.400669,5.181313,38.177120,5.111799,41.585677,5.253243,46.444212
min,66149.0,0.107108,0.015345,0.000953,4.379535e-08,472.741636,8.118813e+02,499.162910,1.183520e+03,658.336276,...,-26.850016,1.325786,-27.809795,1.624544,-20.733809,3.437439,-27.448456,3.065302,-35.640659,0.282131
25%,66149.0,0.315698,0.079833,0.083782,6.145900e-04,1630.680158,1.231961e+05,1887.455790,4.876553e+04,3378.311110,...,-2.227478,29.584894,-7.951722,29.863448,-2.516638,29.636197,-5.734123,30.496412,-4.004475,30.011365
50%,66149.0,0.384741,0.085108,0.121253,1.491318e-03,2208.628236,2.650692e+05,2230.575595,8.996072e+04,4631.377892,...,1.461623,41.702393,-4.443021,42.393583,0.733772,41.831377,-2.702366,43.435253,-1.030939,44.332155
75%,66149.0,0.442443,0.091092,0.176328,3.130862e-03,2712.581884,5.624152e+05,2588.340505,1.585674e+05,5591.634521,...,5.149752,59.274619,-0.726945,61.676964,3.888734,62.033906,0.514246,65.328602,2.216603,68.210421
max,66149.0,0.749481,0.120964,0.442567,3.261522e-02,5432.534406,4.794119e+06,3708.147554,1.235143e+06,9487.446477,...,39.144405,683.932556,34.048843,529.363342,36.970322,629.729797,31.365425,1143.230591,34.212101,910.473206


In [47]:
# Label encoding
class_list = df.iloc[:,-1] 
convert = LabelEncoder()
y = convert.fit_transform(class_list)

# scaling features / Normalization
fit = StandardScaler()
X = fit.fit_transform(np.array(df.iloc[:,:-1],dtype=float))

In [48]:
# dividing into training and test Data
X_train,X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.2)

In [49]:
X_train.shape
# No of features = 58

(7992, 58)

In [50]:
# Dimensionality Reduction using PCA
from sklearn.decomposition import PCA
pca = PCA(.90, svd_solver='full')
pca.fit(X_train)

pca.transform(X_train)
pca.transform(X_test)

array([[-0.6825565 ,  2.17994054, -0.38533985, ...,  0.06152203,
         0.78314678, -0.72786763],
       [-2.38360562, -0.17692208,  1.86197032, ..., -0.84639072,
        -0.31874568,  0.52744133],
       [ 0.7501258 ,  6.6280171 ,  0.45229429, ..., -0.56468707,
         1.298325  , -0.23584881],
       ...,
       [-2.80373402,  1.38871953,  2.18565696, ..., -0.26506562,
         0.15972041, -0.37514006],
       [-5.49281245, -3.85676575,  1.54504393, ..., -0.11833601,
        -0.09321987, -0.12519875],
       [ 1.9764285 ,  1.04217315, -0.39544721, ...,  0.29811411,
         1.23408553,  0.36042031]])

In [51]:
# No of features chosen by PCA such that 95% variance is retained
pca.n_components_

30

In [52]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='sag')
model.fit(X_train, Y_train.reshape(-1,1))
Y_pred = model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [58]:
from sklearn.metrics import f1_score, accuracy_score
print(f1_score(Y_test, Y_pred, average='weighted'))
print(accuracy_score(Y_test, Y_pred))

0.7379699506956575
0.7392392392392393


In [59]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, Y_train.reshape(-1,1))
Y_pred = model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
print(f1_score(Y_test, Y_pred, average='weighted'))
print(accuracy_score(Y_test, Y_pred))

0.8681852318670801
0.8688688688688688


In [61]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, Y_train.reshape(-1,1))
Y_pred = model.predict(X_test)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [62]:
print(f1_score(Y_test, Y_pred, average='weighted'))
print(accuracy_score(Y_test, Y_pred))

0.8705427775138825
0.8708708708708709
